# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы Ультра и Смарт. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75 на тестовой выборке.

## Открыть и изучить файл

In [1]:
import pandas as pd # импорт библиотеки pandas
from sklearn.tree import DecisionTreeClassifier # импорт из библиотеки sklearn алгоритма дерева решений
from sklearn.metrics import accuracy_score # импорт из библиотеки sklearn функции метрики качества
from sklearn.ensemble import RandomForestClassifier # импорт из библиотеки sklearn алгоритма случайного леса
from sklearn.linear_model import LogisticRegression # импорт из библиотеки sklearn алгоритма логистической регрессии
from sklearn.model_selection import train_test_split # импорт из библиотеки sklearn функцию разделения выборок
from sklearn.dummy import DummyClassifier # импорт из библиотеки sklearn алгоритма DummyClassifier

In [2]:
df = pd.read_csv('***.csv') # прочитаем файл
df.info() # общая информация
df.head() # первые пять строк датафрейма

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

В данных нет пропусков. Других проблем тоже не обнаружено. 

Столбец 'is_ultra' - целевой признак. Целевой признак категориальный, значит будем решать задачу классификации.

## Разделение данных на выборки

In [3]:
features = df.drop(['is_ultra'], axis =1) # определяю признаки
target = df['is_ultra'] # определяю целевой признак

Зафиксируем параметр случайного состояния, например random_state=42, для сохранения псевдослучайности алгоритма обучения модели.

In [4]:
features_train, features_test , target_train, target_test = train_test_split(
    features, target, train_size=0.6, random_state=42) # отделение 60% данных для обучающей выборки

In [5]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size=0.5, random_state=42) 
# разделение оставшихся данных на тестовую и валидационную выборки

Проверим разделение выборок:

In [6]:
print('Обучающая выборка:',round(features_train.shape[0]/features.shape[0]*100,1),'%')
print('Валидационная выборка:',round(features_valid.shape[0]/features.shape[0]*100,1),'%')
print('Тестовая выборка:',round(features_test.shape[0]/features.shape[0]*100,1),'%')

Обучающая выборка: 60.0 %
Валидационная выборка: 20.0 %
Тестовая выборка: 20.0 %


Теперь можно приступать к исследованию разных моделей.

## Исследование моделей

### Модель дерева решений.

Создадим цикл с подбором гипермараметра глубины дерева (max_depth). 

Зафиксируем параметр случайного состояния, например random_state=42, для сохранения псевдослучайности алгоритма обучения модели.

In [7]:
best_model = None
best_result = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(random_state=42, max_depth=depth) # модель классификации дерева решений
    
    model.fit(features_train, target_train) # обучение модели
    
    predictions_valid = model.predict(features_valid) # предсказание модели на валидационной выборке
    
    result = accuracy_score(target_valid, predictions_valid) 
    
    if result > best_result:
        best_model = model
        best_result = result
    
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print('Лучшие гиперпараметры:', best_model) 

Accuracy наилучшей модели на валидационной выборке: 0.8118195956454122
Лучшие гиперпараметры: DecisionTreeClassifier(max_depth=4, random_state=42)


In [8]:
model_decision_tree = DecisionTreeClassifier(random_state=42, max_depth=4) # сохранение модели с нужными гиперпараметрами
model_decision_tree.fit(features_train, target_train) # обучили модель с нужными гиперпараметрами

DecisionTreeClassifier(max_depth=4, random_state=42)

### Модель случайного леса.

Создадим цикл с подбором гипермараметра количества деревьев (n_estimators) и глубины дерева (max_depth). 

Так же зададим параметр random_state=42.

In [9]:
best_model = None
best_result = 0
for est in range(1,15):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=42, max_depth=depth, n_estimators=est) # модель классификации дерева решений
    
        model.fit(features_train, target_train) # обучение модели
    
        result = model.score(features_valid, target_valid) # применение модели на валидационной выборке
    
        if result > best_result:
            best_model = model
            best_result = result
        
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print('Лучшие гиперпараметры:', best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8304821150855366
Лучшие гиперпараметры: RandomForestClassifier(max_depth=6, n_estimators=10, random_state=42)


In [10]:
model_random_forest = RandomForestClassifier(random_state=42, max_depth=6, n_estimators=10) # сохранение модели с нужными гиперпараметрами
model_random_forest.fit(features_train, target_train) # обучили модель с нужными гиперпараметрами

RandomForestClassifier(max_depth=6, n_estimators=10, random_state=42)

### Модель логистической регрессии.

Так же зададим параметр random_state=42.

In [11]:
model_logistic = LogisticRegression(max_iter=1000, random_state=42) # модель логистической регрессии
model_logistic.fit(features_train, target_train) # обучение модели
result = model_logistic.score(features_valid, target_valid) # применение модели на валидационной выборке

print('Accuracy модели логистической регрессии на валидационной выборке:', result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7682737169517885


**Вывод**

Наилучший результат показала модель случайного леса, равным 0,83. Следом идет модель дерева решений с метрикой качества 0,812. И замыкает список модель логистической регрессии с результатом 0,768.

У каждой модели есть свои плюсы и минусы. Наилучшее качество как раз показала модель случайного леса, но ее минус в ее медлительности.

Хороший результат показала модель дерева решений, хотя у нее не высокое качество, но высокая скорость. Возможно это зависит от данных.

## Проверка на тестовой выборке

### Модель дерева решений.

In [12]:
predictions_test = model_decision_tree.predict(features_test) # применение модели на тестовой выборке
print('Accuracy модели дерева решений на тестовой выборке:', accuracy_score(target_test, predictions_test))

Accuracy модели дерева решений на тестовой выборке: 0.7807153965785381


### Модель случайного леса.

In [13]:
print('Accuracy модели случайного леса на тестовой выборке:', model_random_forest.score(features_test, target_test))

Accuracy модели случайного леса на тестовой выборке: 0.8040435458786936


### Модель логистической регрессии.

In [14]:
print('Accuracy модели логистической регрессии на тестовой выборке:', model_logistic.score(features_test, target_test))

Accuracy модели логистической регрессии на тестовой выборке: 0.7402799377916018


**Вывод**

Наилучший результат показала модель случайного леса, равным 0,804. Следом идет модель дерева решений с метрикой качества 0,781. И замыкает список модель логистической регрессии с результатом 0,74.

Падение качества результата по моделям в среднем составляет 3%.

Проверка на тестовых данных показывает, что в моделях случайного леса и дерева решений отсутствуют проблемы переобучения и недообучения.

## Проверка модели на адекватность

Чтобы оценить адекватность модели, сравним ее с константной моделью. 

In [15]:
target_test.value_counts(1)

0    0.693624
1    0.306376
Name: is_ultra, dtype: float64

In [16]:
model_dummy = DummyClassifier(strategy='constant', constant=0)
model_dummy.fit(features_train, target_train)
model_dummy.score(features_test, target_test)

0.6936236391912908

Точность константной модели равна доли бОльшего класса.

**Вывод**

Accuracy константной модели равна 0,694. Результаты обученных моделей на тестовых выборках от 0,74 до 0,8.

Можно смело делать вывод, что модели прошли оценку на адекватность.

## Общий вывод.

Я исследовал и сравнил качество следующих моделей на валидационной выборке:
 - Модель дерева решений. Accuracy = 0,812
 - Модель случайного леса. Accuracy = 0,83
 - Модель логистической регрессии.  Accuracy = 0,768
 
Качество моделей на тестовой выборке:
 - Модель дерева решений. Accuracy = 0,781
 - Модель случайного леса. Accuracy = 0,804
 - Модель логистической регрессии.  Accuracy = 0,74

Все модели прошли оценку на адекватность.
 
В цели к проекту указано, что нужно довести долю правильных ответов по крайней мере до 0.75. Две модели справились с этой задачей. Модель логистической регрессии не достаточно точна для этой задачи.

**Наилучший результат accuracy у модели случайного леса. Accuracy = 0,804.**